## PROTOCOL

왜 python에서는 object.length() 대신에 len(object)를 사용하나요?
- 특정한 action에 대해 python 언어가 제시하는 protocol이 있습니다.
- 해당 protocol을 구현하는 어느 객체도 python이 다룰 수 있습니다.
- 기존 상속 메커니즘(혹은 java에서 사용하는 interface)과 다르게 강제되지 않습니다. *필요한 동작에 대한 protocl만 선택해서 구현하면 됩니다.*
- dunder function으로 구현합니다.

In [4]:
# 예) 객체의 길이계산은 __len__을 호출합니다.
class Sample():
    def __len__(self):
        return 10

len(Sample())
    

10

In [25]:
# 예) 호출은 __call__을 호출합니다.
class Sample():
    def __call__(self):
        return 10
sample = Sample()

In [11]:
# 예) in operator는 __contains__를 호출합니다.
class Sample():
    def __contains__(self, any_value):
        return True
sample = Sample()
10 in sample

True

In [12]:
# . operator를 이용한 접근 __getattr__, __setattr__
# [] operator를 이용한 __getitem__, __setitem__

In [14]:
#Lib/UserList.py (python2.7)
"""A more or less complete user-defined wrapper around list objects."""

import collections

class UserList(collections.MutableSequence):
    def __init__(self, initlist=None):
        self.data = []
        if initlist is not None:
            # XXX should this accept an arbitrary sequence?
            if type(initlist) == type(self.data):
                self.data[:] = initlist
            elif isinstance(initlist, UserList):
                self.data[:] = initlist.data[:]
            else:
                self.data = list(initlist)
    def __repr__(self): return repr(self.data)
    def __lt__(self, other): return self.data <  self.__cast(other)
    def __le__(self, other): return self.data <= self.__cast(other)
    def __eq__(self, other): return self.data == self.__cast(other)
    def __ne__(self, other): return self.data != self.__cast(other)
    def __gt__(self, other): return self.data >  self.__cast(other)
    def __ge__(self, other): return self.data >= self.__cast(other)
    def __cast(self, other):
        if isinstance(other, UserList): return other.data
        else: return other
    def __cmp__(self, other):
        return cmp(self.data, self.__cast(other))
    __hash__ = None # Mutable sequence, so not hashable
    def __contains__(self, item): return item in self.data
    def __len__(self): return len(self.data)
    def __getitem__(self, i): return self.data[i]
    def __setitem__(self, i, item): self.data[i] = item
    def __delitem__(self, i): del self.data[i]
    def __getslice__(self, i, j):
        i = max(i, 0); j = max(j, 0)
        return self.__class__(self.data[i:j])
    def __setslice__(self, i, j, other):
        i = max(i, 0); j = max(j, 0)
        if isinstance(other, UserList):
            self.data[i:j] = other.data
        elif isinstance(other, type(self.data)):
            self.data[i:j] = other
        else:
            self.data[i:j] = list(other)
    def __delslice__(self, i, j):
        i = max(i, 0); j = max(j, 0)
        del self.data[i:j]
    def __add__(self, other):
        if isinstance(other, UserList):
            return self.__class__(self.data + other.data)
        elif isinstance(other, type(self.data)):
            return self.__class__(self.data + other)
        else:
            return self.__class__(self.data + list(other))
    def __radd__(self, other):
        if isinstance(other, UserList):
            return self.__class__(other.data + self.data)
        elif isinstance(other, type(self.data)):
            return self.__class__(other + self.data)
        else:
            return self.__class__(list(other) + self.data)
    def __iadd__(self, other):
        if isinstance(other, UserList):
            self.data += other.data
        elif isinstance(other, type(self.data)):
            self.data += other
        else:
            self.data += list(other)
        return self
    def __mul__(self, n):
        return self.__class__(self.data*n)
    __rmul__ = __mul__
    def __imul__(self, n):
        self.data *= n
        return self
    def append(self, item): self.data.append(item)
    def insert(self, i, item): self.data.insert(i, item)
    def pop(self, i=-1): return self.data.pop(i)
    def remove(self, item): self.data.remove(item)
    def count(self, item): return self.data.count(item)
    def index(self, item, *args): return self.data.index(item, *args)
    def reverse(self): self.data.reverse()
    def sort(self, *args, **kwds): self.data.sort(*args, **kwds)
    def extend(self, other):
        if isinstance(other, UserList):
            self.data.extend(other.data)
        else:
            self.data.extend(other)

특정 protocol 구현여부를 검사할 수도 있습니다. (특정한 동작을 수행할 수 있는지)

In [52]:
from collections.abc import Sized, Callable
print(dir(abc))

# 예) 객체의 길이계산은 __len__을 호출합니다.
class Sample:
    def __len__(self):
        return 10
    def __call__(self):
        return 20

sample = Sample()
isinstance(sample, Sized) and len(sample), isinstance(sample, Callable) and sample()

['ABC', 'ABCMeta', 'WeakSet', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', 'abstractclassmethod', 'abstractmethod', 'abstractproperty', 'abstractstaticmethod', 'get_cache_token']


(10, 20)

In [53]:
from collections.abc import Container
class Sample:
    def __contains__(self, _):
        return True

isinstance(Sample(), Container)

True

내가 만든 함수에서 arguement검사를 해본다고 합시다.

In [36]:
def hello(arg):
    assert type(arg) == type({})


hello({'hello': 'world'}) # pass
hello(OrderedDict({'hello': 'world'})) # pass

NameError: name 'OrderedDict' is not defined

In [51]:
from collections.abc import Mapping
from collections import OrderedDict

class MyDict(Mapping):
    def __getitem__(self, key):
        return None
    def __iter__(self):
        pass
    def __len__(self):
        return 0

assert isinstance({'hello': 'world'}, Mapping)
assert isinstance(OrderedDict({'hello': 'world'}), Mapping)
assert isinstance(MyDict(), Mapping)